<a href="https://colab.research.google.com/github/slowvak/MC4-TensorflowUNet/blob/master/Basic_Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cell 1 loads up all the libraries we are using. In the first lessons, we used fastAI, while in this one we use Tensorflow with Keras. Both of these are large libraries, and its designers decided to allow programmers to load only the pieces that are relevant, which is what the 'from keras.XXX import YYY' lines do--they load just the partrt of Keras (or other library) that we want. This makes the load time faster and the final running program smaller.

The second half of this cell does the familiar step of loading the data sets that I have prepared, which in this case are a series of abdominal CTs as well as hand-traced masks of the Pancreas, courtesy of The Cancer Image Archive: http://TheCancerImageArchive....

In [1]:
#Cell 1
# Load the {{PROCESSED}} images and the masks for pancreas from RSNA repository
# To make load times reasonable, only slices with pancreas are included
#  and these are 8 bit JPEG images (compresssed)

!rm -rf ./MC4-TensorflowUNet

!rm -rf images
!mkdir images
!rm -rf masks
!mkdir masks


import os

!git clone https://github.com/slowvak/MC4-TensorflowUNet.git


#for f in os.listdir('./MC4-TensorflowUNet'):
#    cmd = 'unzip ./MC4-TensorflowUNet/{}'.format(f)
#    os.system(cmd)
#    limit = limit - 1
#    if limit < 0:
#        break

!unzip -q -o ./MC4-TensorflowUNet/Pt1.zip
!unzip -q -o ./MC4-TensorflowUNet/Pt2.zip
!unzip -q -o ./MC4-TensorflowUNet/Pt3.zip

!mv *-Mask.jpg ./masks
!mv *.jpg ./images

#!ls images


Cloning into 'MC4-TensorflowUNet'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 109 (delta 7), reused 0 (delta 0), pack-reused 85
Receiving objects: 100% (109/109), 371.49 MiB | 29.59 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Checking out files: 100% (93/93), done.


In [2]:
!pip install -q keras
import keras
from __future__ import print_function
import cv2
import numpy as np

import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose
from keras.layers.merge import concatenate, add
from keras.layers.core import Lambda
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
import numpy
import matplotlib as plt
#import matplotlib.image as mpimg
import imageio



Using TensorFlow backend.


In [3]:
# Get training and testing data
# This Block loads the training images and training masks into numpy arrays of shape (n, 256, 256)


train_X = []
train_Y = []
test_X = []
test_Y = []


limit = 3500  # limit number of subjects due to memory limits
test_num = 250  # will also load some test images for the end

im_list = os.listdir('./images')

for f in im_list:
  pth = './images/' + f 
  img = imageio.imread(pth)
  if len(train_X) >= limit:
    test_X.append(np.array(img))
  else:
    train_X.append(np.array(img))
  if len(test_X) >= test_num:
    break

im_list = os.listdir('./masks')
for f in im_list:
  pth = './masks/' + f 
  img = imageio.imread(pth)
  if len(train_Y) >= limit:
    test_Y.append(np.array(img))
  else:
    train_Y.append(np.array(img))
  if len(test_Y) >= test_num:
    break

X_array = numpy.array(train_X)
Y_array = numpy.array(train_Y)  
# the 2 lines below adds an axis, which is needed by Keras
X_array = X_array[..., np.newaxis]
Y_array = Y_array[..., np.newaxis]

X_test = numpy.array(test_X)
Y_test = numpy.array(test_Y)                                            
X_test = X_test[..., np.newaxis]
Y_test = Y_test[..., np.newaxis]

X_array.shape
CHANNELS = X_array.shape[3]
IMG_WIDTH = X_array.shape[2]
IMG_HEIGHT = X_array.shape[1]
NUM_IMAGES = X_array.shape[0]
K.set_image_dim_ordering('tf')

print (str(NUM_IMAGES) + ' for training and validation, and testing has ' + str(X_test.shape[0]))


3500 for training and validation, and testing has 250


# Cost Functions
A critical element of every machine learning algorithm is to select a good cost function. In the classificaiton problems of prior articles, we used the accuracy of predictions as the cost function. For segmentation, one potentially could compute a score for teh classification of each pixel (object or not object), but a more common metric is the Dice Similiarity Score (a.k.a. Dice Score). This essentially means the amount of overlap between the gold standard and the prediction. If they perfectly agree, the Dice Score is 1, and if there is no overlap, the score is 0. We can convert the Dice Score toa Dice Loss by subtracting it from 1. One can also convert the Dice Score to a cross-entropy function, and this can work well in some cases. Code for both is provided, and you are encouraged to try both cost functions to see the effect.
Note, however, that the above can result in Dice values of zero, and we don't want to divide by zero. In fact, we don't want to divide by a small number, so we add 'smooth' to the value (in our case we are using 1) so the values of Dice actually are 1 to 2, not 0 to 1. And that means the Dice Loss will be -1 to 0, but the loss reported is the sum, so it will have a larger range.

In [0]:
# Cell 
tf.reset_default_graph()

def dice_coeff(y_true, y_pred):
    smooth = 0.000001
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    if y_true_f + y_pred_f == 0:
        return 4 * smooth
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

#binary cross entropy is another function that can perform well
def bce_dice_loss(y_true, y_pred):
    loss = losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

##Building the U-Net
We are finally ready to build our Network. One difference of Keras versus the previous examples of FastAI is that Keras is designed so that there is one line of code for each layer of the network (plus the setup and training/evlauation code).  This means that for a moderately complex network like a ResNet34 classifier, there would be 34 lines of code. 
U-Nets are popular network architectures for segmentation. These get their name because of the unique way they are built: The first part produces repeated reductions in resolution as the 'important' parts of the image are retained that reflect the structures that it is trained to recognize. Typically there are 4 or 5 such reduction layers that each consist of convultions, ReLUs, and Pooling layers. 
Once the image is reduced to the critical components, the networks begins to reconstruct the precise margins of the critical elements by using 'skip layers'. As the resolution is stored using convolutional-transpose layers, the layers 'look' back to the layers where the resolution was reduced to try to best define the margins of the structures.
See Ronneberger...

In [0]:
# Cell 


# Build U-Net model
def build_model(drop_out = 0.0, act_fn = 'relu', init_fn = 'he_normal'):

    inputs = Input((IMG_HEIGHT, IMG_WIDTH, CHANNELS))
    s = Lambda(lambda x: x / 255) (inputs)

    c1 = Conv2D(32, (7, 7), activation=act_fn, kernel_initializer=init_fn, padding='same') (s)
    #c1 = Conv2D(32, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (s)
    if drop_out > 0.0:
        c1 = Dropout(drop_out) (c1)
        c1 = Conv2D(32, (7, 7), activation=act_fn, kernel_initializer=init_fn, padding='same') (c1)
    p1 = MaxPooling2D(pool_size=(2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (p1)
    if drop_out > 0.0:
        c2 = Dropout(drop_out) (c2)
        c2 = Conv2D(32, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (p2)
    if drop_out > 0.0:
        c3 = Dropout(drop_out) (c3)
        c3 = Conv2D(64, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(128, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (p3)
    if drop_out > 0.0:
        c4 = Dropout(drop_out) (c4)
        c4 = Conv2D(128, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(256, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (p4)
    if drop_out > 0.0:
        c5 = Dropout(drop_out) (c5)
        c5 = Conv2D(256, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (u6)
    if drop_out > 0.0:
        c6 = Dropout(drop_out) (c6)
        c6 = Conv2D(128, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (u7)
    if drop_out > 0.0:
        c7 = Dropout(drop_out) (c7)
        c7 = Conv2D(64, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (u8)
    if drop_out > 0.0:
        c8 = Dropout(drop_out) (c8)
        c8 = Conv2D(32, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (u9)
    if drop_out > 0.0:
        c9 = Dropout(drop_out) (c9)
        c9 = Conv2D(16, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    #model.compile(optimizer='adam'(lr=0.0001,decay=0.000001), loss='dice_loss', metrics=[dice_coeff])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coeff])
#    model.summary()
    return model



# Build U-Net model
def build_shallow_model(drop_out = 0.0, act_fn = 'relu', init_fn = 'he_normal'):

    inputs = Input((IMG_HEIGHT, IMG_WIDTH, CHANNELS))
    s = Lambda(lambda x: x / 255) (inputs)

    c1 = Conv2D(32, (7, 7), activation=act_fn, kernel_initializer=init_fn, padding='same') (s)
    #c1 = Conv2D(32, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (s)
    if drop_out > 0.0:
        c1 = Dropout(drop_out) (c1)
        c1 = Conv2D(32, (7, 7), activation=act_fn, kernel_initializer=init_fn, padding='same') (c1)
    p1 = MaxPooling2D(pool_size=(2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (p1)
    if drop_out > 0.0:
        c2 = Dropout(drop_out) (c2)
        c2 = Conv2D(32, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (p2)
    if drop_out > 0.0:
        c3 = Dropout(drop_out) (c3)
        c3 = Conv2D(64, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)
    
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c3)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (u8)
    if drop_out > 0.0:
        c8 = Dropout(drop_out) (c8)
        c8 = Conv2D(32, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (u9)
    if drop_out > 0.0:
        c9 = Dropout(drop_out) (c9)
        c9 = Conv2D(16, (3, 3), activation=act_fn, kernel_initializer=init_fn, padding='same') (c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    #model.compile(optimizer='adam'(lr=0.0001,decay=0.000001), loss='dice_loss', metrics=[dice_coeff])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coeff])
#    model.summary()
    return model


In [6]:
!pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab
tb = TensorBoardColab()



Wait for 8 seconds...
TensorBoard link:
https://95415872.ngrok.io


# Train it
Now that our model is built, we are (finally!) ready to train it. Trial and error with colab hardware (K80) has shown that a batch_size of 16 works, and that we can get reasonable results adfter 5 epochs (but you can train longer to get better results)

In [7]:

epochs = 50
# more than 50 produces overfitting--train performance keeps incrasing while validation performance starts dropping--try 100 epochs
batch_size = 64


# code to use tensorboard

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -fq ngrok-stable-linux-amd64.zip

# start the tensorboard app
!mkdir ./log
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# see results on port 6006
get_ipython().system_raw('./ngrok http 6006 &')

# then make access available to your computer
# this will create an URL for you to click on to see the training
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

        
# Also need to make a callback--a way to have the training routine post its results to Tensorboard
from keras.callbacks import TensorBoard
tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

#Must add "callbacks=[tbCallBack]" (without the quotes) to the model.fit call


# Fit model
checkpointer = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

#for dropout in [0.0, 0.2, 0.5]:
for dropout in [0.0]:
    print ('\n\nDropout = ' + str(dropout))
    model = build_shallow_model(drop_out = dropout)
    # adding dropout takes 2x time, so allow for twice as many epochs to get about equal execution time
    if dropout < 0.05:
        epoch_cnt = epochs * 2
    else:
        epoch_cnt = epochs
    results = model.fit(X_array, Y_array, validation_split=0.2, batch_size=batch_size, epochs=epoch_cnt, callbacks=[tbCallBack])

--2019-08-23 17:30:48--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.232.40.183, 52.201.75.180, 52.207.111.186, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.232.40.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  14.5MB/s    in 0.9s    

2019-08-23 17:30:49 (14.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

https://95415872.ngrok.io


W0823 17:30:55.009787 140188106078080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0823 17:30:55.016321 140188106078080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0823 17:30:55.028056 140188106078080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0823 17:30:55.057971 140188106078080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0823 17:30:55.089307 140188106078080 deprec



Dropout = 0.0


W0823 17:30:55.313208 140188106078080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0823 17:30:55.337427 140188106078080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0823 17:30:55.343024 140188106078080 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2800 samples, validate on 700 samples


W0823 17:30:59.169125 140188106078080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:848: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W0823 17:30:59.394568 140188106078080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:840: The name tf.summary.image is deprecated. Please use tf.compat.v1.summary.image instead.

W0823 17:31:02.609343 140188106078080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0823 17:31:02.612603 140188106078080 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/100
2800/2800 [==============================] - 39s 14ms/step - loss: 0.5230 - dice_coeff: 0.5733 - val_loss: 0.3212 - val_dice_coeff: 0.6475
Epoch 2/100
2800/2800 [==============================] - 29s 10ms/step - loss: 0.3218 - dice_coeff: 0.7469 - val_loss: 0.3329 - val_dice_coeff: 0.6901


KeyboardInterrupt: ignored

In [0]:
#FastAI version of UNet
from fastai.vision import *



# first, prepare the DataBunch
path_msk = './masks/'
path_img = './images/'

def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

fnames = listdir_fullpath(path_img)
lbl_names = listdir_fullpath(path_msk)

codes = ['NotTumor', 'Tumor']

# open and show image
img_f = fnames[10]
img = open_image(img_f)
img.show(figsize=(5, 5))

get_mask_file_fn = lambda x: os.path.join(path_msk,os.path.basename(x).replace ('.jpg', '-Mask.jpg'))

size = 400
batch_size = 32

src = (SegmentationItemList.from_folder(path_img)
       # Load in x data from folder
       .split_by_rand_pct(valid_pct = 0.2, seed=None)
       .label_from_func(get_mask_file_fn, classes=codes)
       # Label data using the get_y_fn function
)

data = (src.transform(get_transforms(), size=size, tfm_y=True)
        # Flip images horizontally 
        .databunch(bs=batch_size)
        #.normalize(imagenet_stats)
        # Normalize for resnet
)

In [0]:


learn = unet_learner(data, models.resnet18, metrics=dice, wd=1e-2)


learn.lr_find() # find learning rate
learn.recorder.plot() # plot learning rate graph


In [0]:
lr = 3e-3 # pick a lr
learn.fit_one_cycle(10, slice(lr), pct_start=0.9) # train model

learn.save('LGG') # save model
learn.show_results(rows=3, figsize=(8, 9)) # show results


learn.unfreeze() # unfreeze all layers

# find and plot lr again
learn.unfreeze()
learn.recorder.plot()

# train model 
learn.fit_one_cycle(12, slice(lr/400, lr/4), pct_start=0.8)




In [13]:
# from https://github.com/hlamba28/UNET-TGS/blob/master/TGS%20UNET.ipynb

def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
    """Function to define the UNET Model"""
    # Contracting Path
    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
    
    # Expansive Path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

input_img = Input((IMG_HEIGHT, IMG_WIDTH, CHANNELS), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])



callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

results = model.fit(X_array, Y_array, validation_split=0.2, batch_size=batch_size, epochs=epoch_cnt, callbacks=callbacks)
#results = model.fit(X_train, y_train, batch_size=32, epochs=50, callbacks=callbacks,\
#                    validation_data=(X_valid, y_valid))

Train on 2800 samples, validate on 700 samples
Epoch 1/100
2800/2800 [==============================] - 65s 23ms/step - loss: 0.2532 - acc: 0.6581 - val_loss: -0.0260 - val_acc: 0.6476

Epoch 00001: val_loss improved from inf to -0.02603, saving model to model.h5
Epoch 2/100
2800/2800 [==============================] - 50s 18ms/step - loss: -0.0376 - acc: 0.6974 - val_loss: -0.6554 - val_acc: 0.6434

Epoch 00002: val_loss improved from -0.02603 to -0.65538, saving model to model.h5
Epoch 3/100
2800/2800 [==============================] - 50s 18ms/step - loss: -0.1335 - acc: 0.7035 - val_loss: -0.7283 - val_acc: 0.7547

Epoch 00003: val_loss improved from -0.65538 to -0.72835, saving model to model.h5
Epoch 4/100
2800/2800 [==============================] - 50s 18ms/step - loss: -0.3893 - acc: 0.7338 - val_loss: 1.0026 - val_acc: 0.5293

Epoch 00004: val_loss did not improve from -0.72835
Epoch 5/100
2800/2800 [==============================] - 50s 18ms/step - loss: -0.5529 - acc: 0.695

KeyboardInterrupt: ignored

In [0]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

# Test it!
Now that the model is trained, we can test our segmentation tool on the test holdout cases. Note that the prediction is the probability that a pixel is object (pancreas) or not object (anything other than pancreas). We use 0.5 as the threshold for deciding Pancreas or not, though again you can adjust this if your task might view it as more valuable to include non-pancreas pixels in order to reduce the number of missed pancreas pixels.

In [0]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt

# Predict on train, val and test
#model = load_model('model.h5', custom_objects={'dice_loss': dice_loss, 'dice_coeff': dice_coeff})

#preds_train = model.predict(X_array[:int(X_array.shape[0]*0.9)], verbose=1)
#preds_val = model.predict(X_array[int(X_test.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
#preds_train_t = (preds_train > 0.5).astype(np.uint8)
#preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
#preds_test_upsampled = []
#for i in range(len(preds_test)):
#    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True))

fig, axs = plt.subplots(1, 9, figsize=(21, 3))
axs[0].imshow(np.squeeze(X_test[0]), cmap=cm.gray)
axs[1].imshow(np.squeeze(Y_test[0]))
axs[2].imshow(np.squeeze(preds_test_t[0]))
axs[3].imshow(np.squeeze(X_test[4]), cmap=cm.gray)
axs[4].imshow(np.squeeze(Y_test[4]))
axs[5].imshow(np.squeeze(preds_test_t[4]))
axs[6].imshow(np.squeeze(X_test[9]), cmap=cm.gray)
axs[7].imshow(np.squeeze(Y_test[9]))
axs[8].imshow(np.squeeze(preds_test_t[9]))

plt.show()
